In [1]:
from matplotlib import pyplot
from matplotlib.image import imread
import matplotlib.pyplot as plt

In [2]:
import seaborn as sns

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, Dense, Flatten, InputLayer, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.python.framework.ops import disable_eager_execution
#disable_eager_execution()
#tf.compat.v1.enable_eager_execution()

In [4]:
import numpy as np
import pandas as pd

In [5]:
from google.colab import drive

In [6]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
pwd

'/content'

In [8]:
cd drive/"My Drive"/Colab_Notebooks

/content/drive/My Drive/Colab_Notebooks


In [9]:
batch_size = 32

GPU Resources: https://www.tutorialspoint.com/google_colab/google_colab_using_free_gpu.htm
https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/gpu.ipynb#scrollTo=IqR2PQG4ZaZ0

In [10]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [11]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [12]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14095327118195292814
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 16139419648
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12047108892491827236
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
 xla_global_id: 416903419]

In [13]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Create 2 virtual GPUs with 1GB memory each
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024),
         tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPU, 2 Logical GPUs


In [14]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():
  inputs = tf.keras.layers.Input(shape=(1,))
  predictions = tf.keras.layers.Dense(1)(inputs)
  model = tf.keras.models.Model(inputs=inputs, outputs=predictions)
  model.compile(loss='mse',
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.2))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [15]:
tf.debugging.set_log_device_placement(True)

gpus = tf.config.list_logical_devices('GPU')
if gpus:
  # Replicate your computation on multiple GPUs
  c = []
  for gpu in gpus:
    with tf.device(gpu.name):
      a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
      b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
      c.append(tf.matmul(a, b))

  with tf.device('/CPU:0'):
    matmul_sum = tf.add_n(c)

  print(matmul_sum)

tf.Tensor(
[[ 44.  56.]
 [ 98. 128.]], shape=(2, 2), dtype=float32)


### Augmentation Configurations

In [16]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [17]:
test_datagen = ImageDataGenerator(rescale=1./255)

### Generator Configuration

In [18]:
train_generator = train_datagen.flow_from_directory(
        'dataset_dogs_vs_cats/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical') 

Found 18819 images belonging to 2 classes.


In [19]:
test_generator = test_datagen.flow_from_directory(
        'dataset_dogs_vs_cats/test',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False) 

Found 6355 images belonging to 2 classes.


In [20]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [21]:
from tensorflow.keras.applications import inception_v3
inception_conv = inception_v3.InceptionV3(weights=None, include_top=False)
inception_conv.summary()
inputs = Input(shape=(150,150,3),name = 'image_input')  # our input shape: (32,32,3)
output_inception = inception_conv(inputs) # using functional api syntax
#Add the fully-connected layers 
x = Flatten(name='flatten')(output_inception)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(2048, activation='relu')(x)
x = Dense(2, activation='softmax', name='predictions')(x)
my_model = Model(inputs=inputs, outputs=x)  # Using the Functional API

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   864         ['input_2[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   96         ['conv2d[0][0]']                 
 alization)                     32)                                                    

In [22]:
inputs = Input(shape=(150,150,3,1),name = 'image_input')  # our input shape: (32,32,3)

In [23]:
import math
from math import floor

In [24]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [25]:
my_model.compile(optimizer='SGD', loss='categorical_crossentropy',metrics=['accuracy', 'mse',f1_m,precision_m, recall_m])

In [26]:
history = my_model.fit_generator(train_generator, 
                        epochs=20, 
                        verbose=True, 
                        callbacks=[keras.callbacks.EarlyStopping(patience=8, verbose=1,restore_best_weights=True),
                              keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)],
                       shuffle=True
                        )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/20
589/589 [==============================] - 1826s 3s/step - loss: 0.7216 - accuracy: 0.5614 - mse: 0.2509 - f1_m: 0.5621 - precision_m: 0.5621 - recall_m: 0.5621 - lr: 0.0100
Epoch 2/20
589/589 [==============================] - 160s 271ms/step - loss: 0.6494 - accuracy: 0.6227 - mse: 0.2286 - f1_m: 0.6222 - precision_m: 0.6222 - recall_m: 0.6222 - lr: 0.0100
Epoch 3/20
589/589 [==============================] - 160s 272ms/step - loss: 0.6126 - accuracy: 0.6684 - mse: 0.2117 - f1_m: 0.6674 - precision_m: 0.6674 - recall_m: 0.6674 - lr: 0.0100
Epoch 4/20
589/589 [==============================] - 159s 271ms/step - loss: 0.5807 - accuracy: 0.6954 - mse: 0.1982 - f1_m: 0.6958 - precision_m: 0.6958 - recall_m: 0.6958 - lr: 0.0100
Epoch 5/20
589/589 [==============================] - 160s 271ms/step - loss: 0.5456 - accuracy: 0.7205 - mse: 0.1842 - f1_m: 0.7209 - precision_m: 0.7209 - recall_m: 0.7209 - lr: 0.0100
Epoch 6/20
589/589 [==============================] - 160s 271ms/st

In [27]:
prediction_list = my_model.predict_generator(test_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import confusion_matrix


In [ ]:
len(prediction_list)

In [ ]:
prediction_list = my_model.predict_generator(test_generator)

In [ ]:
print(prediction_list[:10])

In [ ]:
#sum(1 for _ in test_generator)

In [ ]:
def make_confusion_matrix(model, prediction_list, threshold=0.5):
    # Predict class 1 if probability of being in class 1 is greater than threshold
    # (model.predict(X_test) does this automatically with a threshold of 0.5)
    #pred_labels = [lambda x: x for x in prediction_list if x >= threshold]
    y_labels = test_generator.classes
    fraud_confusion = confusion_matrix(y_labels, new_pred_labels)
    plt.figure(dpi=80)
    sns.heatmap(fraud_confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt='d',
           xticklabels=['cat', 'dog'],
           yticklabels=['cat', 'dog']);
    plt.xlabel('prediction')
    plt.ylabel('actual')


In [ ]:
make_confusion_matrix(my_model,prediction_list)

In [ ]:
print(prediction_list[:10])

In [ ]:
actual_labels = test_generator.classes


In [ ]:
print(actual_labels[:10])

In [ ]:
pred_labels = my_model.predict(test_generator)

In [ ]:
print(pred_labels[:10])

In [ ]:
new_pred_labels = []
for item in pred_labels:
  if item[0] > item[1]:
    new_pred_labels.append(0)
  else:
    new_pred_labels.append(1)


In [ ]:
print(new_pred_labels[:10])

In [ ]:
pred_labels = prediction_list >= 0.5

In [ ]:
print(pred_labels[:10])

In [ ]:
incorrects = np.nonzero(pred_labels != actual_labels)

In [ ]:
print(incorrects)

In [ ]:
indices = [i for i,v in enumerate(new_pred_labels) if new_pred_labels[i] != actual_labels[i]]
subset_of_wrongly_predicted = [new_pred_labels[i] for i in indices]

In [ ]:

print(indices)

In [ ]:
print(len(indices))

folder = '**test**/"

In [ ]:
pwd

In [ ]:
folder = 'test1/'

In [ ]:
def load_image(index):
  filename = folder + str(index+1)+'.jpg'
  image = imread(filename)
  pyplot.imshow(image)

In [ ]:
load_image(6)